## Imports

In [56]:
import pyxdf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lmfit.models import Model
from os import listdir, getcwd
from os.path import isfile, join

## Function Definitions


Predicted pupil dilation, $d(Y)$, caused by luminance $Y$, is computed with the following equation: $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$

In [57]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [58]:
def import_data(file):
    streams, header = pyxdf.load_xdf(file)
    dfs = {}
    for stream in streams:
        stream_name = stream['info']['name'][0]
        stream_channels = {}
        for i, channels in enumerate(stream['info']['desc'][0]['channels'][0]['channel']):
            stream_channels[channels['label'][0]] = i
        stream_data = stream['time_series']
        data_dict= {}
        for key in stream_channels.keys():
            data_dict[key] = np.array(stream_data)[:, stream_channels[key]]
        time_stamps = np.array(stream['time_stamps'])
        for i in range(len(time_stamps)):
            time_stamps[i] = round(time_stamps[i], 4)
        data_dict['time'] = time_stamps
        dfs[stream_name] = pd.DataFrame(data_dict)
        dfs[stream_name].drop_duplicates(subset=['time'], keep='first', inplace=True)
        dfs[stream_name].reset_index(drop=True, inplace=True)

## Import Data

In [59]:
data_dir = join(getcwd(),'Path_Data')
data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f))]

In [60]:
dfs = {}
for i, file in enumerate(data_files):
    dfs[i] = import_data(file)